# Day 3: Crossed Wires

https://adventofcode.com/2019/day/3

Two **wires** are connected to a **central port** and extend outward on a **grid**. 

You trace the path each wire takes as it leaves the central port, one wire per line of text.

## Commands

If the first wire's path is `R8,U5,L5,D3`, then starting from the central port (o), 

it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

In [64]:
module Command =
    
    type Distance = int
    
    type Direction =
        | U
        | D
        | L
        | R
        static member ofChar c : Direction =
            match c with
            | 'U' -> U
            | 'D' -> D
            | 'L' -> L
            | 'R' -> R
            | X -> failwith (sprintf "Invalid direction %A" X) 
    
    type Command = 
        | Command of Direction * Distance
        static member ofString (s:String) : Command = 
            
            let (dir,dist) = 
                match s |> Seq.toList with
                | dir::[dist] -> (Direction.ofChar dir, Int32.Parse (string dist))
                |  _ -> failwith (sprintf "Invalid command string %A" s)  
                    
            Command ( dir , dist )
    
    let parse (input:String) : Command seq =
        input.Split(",") 
        |> Seq.ofArray
        |> Seq.map Command.ofString


"R8,U5,L5,D3" |> Command.parse |> Seq.map (string)

index,value
0,"Command (R,8)"
1,"Command (U,5)"
2,"Command (L,5)"
3,"Command (D,3)"


# Problem

The wires twist and turn, but the two wires occasionally cross paths. 

To fix the circuit, you need to find the intersection point closest to the central port. 

Because the wires are on a grid, use the [Manhattan distance](https://en.wikipedia.org/wiki/Taxicab_geometry) for this measurement. 

While the wires do technically cross right at the central port where they both start, 

this point does not count, nor does a wire count as crossing with itself.

# A couple wires

Since we are working on a grid, we can consider a wire a collection of points.

The intersection of two wires are the points they have in common.

- To compute all the `Point`s that make a `Wire`, we will use a `Cursor` that will 
follow the provided `Commands`.

- In order to simplify the `Cursor` behavior, the `Commands` will expanded to perform single steps.

In [65]:
open Command

type Point = 
    | Point of int*int

// extend Command definition

type Command with
    // return a sequence of single steps (as Direction)
    static member AsDirections (c:Command) : Direction seq =
        let (Command (dir, dist)) = c
        seq { for i in 1 .. dist -> dir }

type Cursor(p:Point) =
    let mutable position = p
    member self.Move (d:Direction) =
        let (Point (x,y)) = position
        match d with
        | U -> position <- Point(x,y+1)
        | D -> position <- Point(x,y-1)
        | L -> position <- Point(x-1,y)
        | R -> position <- Point(x+1,y)
    member self.Position with get() = position

let generatePoints (commands:Command seq) : Point seq =
    
    // create a cursor at the start position
    
    let cursor = Cursor (Point (0,0))
    
    // return a sequence ...
    
    seq {
        
        // ... with first item being the central point
        
        yield cursor.Position
            
        // .. followed by
    
        for command in commands do
            
            // .. sequence of points each command makes the cursor visit
            
            yield!  command
                    |> Command.AsDirections // convert to multiple single steps (as a sequence of Direction)
                    |> Seq.map ( fun d -> 
                                    do cursor.Move d   // move cursor one step as specified direction
                                    cursor.Position    // return the position the cursor it's at
                    ) //each position the cursor visits
    }


# Example

If the first wire's path is `R8,U5,L5,D3`, then starting from the central port (o), it lands at point **(3,2)**.

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```


In [68]:
let example_wire_1 = "R8,U5,L5,D3"

example_wire_1 |> Command.parse |> generatePoints |> Seq.rev |> Seq.head |> (string)

Point (3,2)

Then, if the second wire's path is `U7,R6,D4,L4`, it goes up 7, right 6, down 4, and left 4,
landing at **(2,3)**

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........

```

In [70]:
let example_wire_2 = "U7,R6,D4,L4"

example_wire_2 |> Command.parse |> generatePoints |> Seq.rev |> Seq.head |> (string)

Point (2,3)